In [1]:
# Python 2.7

import math, random
from PIL import Image, ImageDraw
import numpy as np
from random import randint

In [2]:

def generatePolygon( ctrX, ctrY, aveRadius, irregularity, spikeyness, numVerts ) :
    '''Start with the centre of the polygon at ctrX, ctrY, 
    then creates the polygon by sampling points on a circle around the centre. 
    Randon noise is added by varying the angular spacing between sequential points,
    and by varying the radial distance of each point from the centre.

    Params:
    ctrX, ctrY - coordinates of the "centre" of the polygon
    aveRadius - in px, the average radius of this polygon, this roughly controls how large the polygon is, really only useful for order of magnitude.
    irregularity - [0,1] indicating how much variance there is in the angular spacing of vertices. [0,1] will map to [0, 2pi/numberOfVerts]
    spikeyness - [0,1] indicating how much variance there is in each vertex from the circle of radius aveRadius. [0,1] will map to [0, aveRadius]
    numVerts - self-explanatory

    Returns a list of vertices, in CCW order.
    '''

    irregularity = clip( irregularity, 0,1 ) * 2*math.pi / numVerts
    spikeyness = clip( spikeyness, 0,1 ) * aveRadius

    # generate n angle steps
    angleSteps = []
    lower = (2*math.pi / numVerts) - irregularity
    upper = (2*math.pi / numVerts) + irregularity
    sum = 0
    for i in range(numVerts) :
        tmp = random.uniform(lower, upper)
        angleSteps.append( tmp )
        sum = sum + tmp

    # normalize the steps so that point 0 and point n+1 are the same
    k = sum / (2*math.pi)
    for i in range(numVerts) :
        angleSteps[i] = angleSteps[i] / k

    # now generate the points
    points = []
    angle = random.uniform(0, 2*math.pi)
    for i in range(numVerts) :
        r_i = clip( random.gauss(aveRadius, spikeyness), 0, 2*aveRadius )
        x = ctrX + r_i*math.cos(angle)
        y = ctrY + r_i*math.sin(angle)
        points.append( (int(x),int(y)) )

        angle = angle + angleSteps[i]

    return points

def clip(x, min, max) :
    if( min > max ):  return x
    elif( x < min ):  return min
    elif( x > max ):  return max
    else:             return x

In [17]:
# Drawing single polygons
verts = generatePolygon(14, 14, 8, 0.5, 0, 2)

black = (0,0,0)
white=(255,255,255)
im = Image.new('L', (28, 28), 0)
imPxAccess = im.load()
draw = ImageDraw.Draw(im)
tupVerts = map(tuple,verts)

# either use .polygon(), if you want to fill the area with a solid colour
draw.polygon( tupVerts, outline=255,fill=0 )

# or .line() if you want to control the line thickness, or use both methods together!
# draw.line( tupVerts+[tupVerts[0]], width=2, fill=black )


## Generating various datasets to test learned statistical features

In [158]:
# generate dataset - centered polygons, 28x28 images
for i in [3,4,5]:
    triangles=np.zeros((784,1000))
    for j in range(0,1000):
        verts = generatePolygon(14, 14, 11, 0.5, 0, i)
        im = Image.new('L', (28, 28), 0)
        imPxAccess = im.load()
        draw = ImageDraw.Draw(im)
        tupVerts = map(tuple,verts)
        draw.polygon( tupVerts, outline=255,fill=255 )
        triangles[:,i] = np.array(im.getdata())
        #im.save('test2','tiff')
        

In [180]:
# Generate Datasets V1
pentagons=np.zeros((784,1000))
for j in range(0,1000):
    verts = generatePolygon(15, 15, 11, 0.5, 0, 5)
    im = Image.new('L', (28, 28), 0)
    imPxAccess = im.load()
    draw = ImageDraw.Draw(im)
    tupVerts = map(tuple,verts)
    draw.polygon( tupVerts, outline=255,fill=255 )
    pentagons[:,j] = np.array(im.getdata())
    im.save('image_' + str(j+2000) + '.png')
        

In [181]:
np.savetxt("triangles", triangles)
np.savetxt("tetragons", tetragons)
np.savetxt("pentagons", pentagons)

In [74]:
# Generate NEW Datasets with various object areas V2
pentagons1=np.zeros((2025,1000))
for j in range(0,1000):
    verts = generatePolygon(23, 23, np.random.randint(11,17), 0.5, 0, 5)
    im = Image.new('L', (45, 45), 0)
    imPxAccess = im.load()
    draw = ImageDraw.Draw(im)
    tupVerts = map(tuple,verts)
    draw.polygon( tupVerts, outline=255,fill=255 )
    pentagons1[:,j] = np.array(im.getdata())
    im.save('image_' + str(j+4000) + '.png')
    
#randint(13,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [7]:
# Generate Datasets PolygonImages V3 distribution more overlapped compared to V2
tempObject=np.zeros((2025,1000))
for j in range(0,1000):
    verts = generatePolygon(23, 23, np.random.randint(14,21), 0.5, 0, 3)
    im = Image.new('L', (45, 45), 255)
    imPxAccess = im.load()
    draw = ImageDraw.Draw(im)
    tupVerts = map(tuple,verts)
    draw.polygon( tupVerts, outline=0,fill=0 )
    tempObject[:,j] = np.array(im.getdata())
    im.save('image_' + str(j+1000) + '.png')

#Triangles1 = tempObject

#randint(14,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [16]:
# Generate Datasets PolygonImages V4 translational invariance
# This can be useful to figre out convNet from fullyconnected Network
tempObject=np.zeros((3600,1000))
a=0
b=255
for j in range(0,1000):
    verts = generatePolygon(30+np.random.randint(-8,8), 30+np.random.randint(-8,8), np.random.randint(11,17), 0.5, 0, 3)
    im = Image.new('L', (60, 60), a)
    imPxAccess = im.load()
    draw = ImageDraw.Draw(im)
    tupVerts = map(tuple,verts)
    draw.polygon( tupVerts, outline=b,fill=a )
    tempObject[:,j] = np.array(im.getdata())
    im.save('image_' + str(j+0) + '.png')

#Triangles1 = tempObject

#randint(14,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [43]:
# V5 just more number of images compare to V4 and filled.
# Generate Datasets PolygonImages V5 translational invariance.
# *** This can be useful to figre out convNet from fullyconnectedNet ***
r=[14,21,12,19,11,17]
c=0
cc=0
for x in range(3,6):
    for a in [0,255]:
        
        tempObject=np.zeros((3600,3000))
        if a==0:
            b=255
        else:
            b=0
        
        for j in range(0,3000):
            verts = generatePolygon(30+np.random.randint(-8,8), 30+np.random.randint(-8,8), np.random.randint(r[cc],r[cc+1]), 0.5, 0, x)
            im = Image.new('L', (60, 60), a)
            imPxAccess = im.load()
            draw = ImageDraw.Draw(im)
            tupVerts = map(tuple,verts)
            draw.polygon( tupVerts, outline=b,fill=b )
            tempObject[:,j] = np.array(im.getdata())
            im.save('image_' + str(j+c*3000) + '.png')
        
        c = c+1
    cc = cc+2

#Triangles1 = tempObject

#randint(14,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [ ]:
# V6 just V5 but not filled 
# Generate Datasets PolygonImages V5 translational invariance - not generated yet
# *** This can be useful to figre out convNet from fullyconnectedNet ***
tempObject=np.zeros((3600,3000))
a=255
b=0
for j in range(0,3000):
    verts = generatePolygon(30+np.random.randint(-8,8), 30+np.random.randint(-8,8), np.random.randint(11,17), 0.5, 0, 5)
    im = Image.new('L', (60, 60), a)
    imPxAccess = im.load()
    draw = ImageDraw.Draw(im)
    tupVerts = map(tuple,verts)
    draw.polygon( tupVerts, outline=b,fill=a )
    tempObject[:,j] = np.array(im.getdata())
    im.save('image_' + str(j+15000) + '.png')

#Triangles1 = tempObject

#randint(14,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [ ]:
# Generate Dataset V7 centered - no inverse - big dataset. modified from V5

r=[14,21,12,19,11,17]
c=0
cc=0
for x in range(3,6):
        
    tempObject=np.zeros((2025,20000))
    for j in range(0,20000):
        verts = generatePolygon(23, 23, np.random.randint(r[cc],r[cc+1]), 0.5, 0, x)
        im = Image.new('L', (45, 45), 0)
        imPxAccess = im.load()
        draw = ImageDraw.Draw(im)
        tupVerts = map(tuple,verts)
        draw.polygon( tupVerts, outline=255,fill=255 )
        tempObject[:,j] = np.array(im.getdata())
        im.save('image_' + str(j+c*20000) + '.png')

    c = c+1
    cc = cc+2

#Triangles1 = tempObject

#randint(14,21) for trianges
#randint(12,19) for tetragons
#randint(11,17) for pentagons

In [ ]:
np.savetxt("triangles1", triangles1)
np.savetxt("tetragons1", tetragons1)
np.savetxt("pentagons1", pentagons1)
np.savetxt("triangles2", triangles2)
np.savetxt("tetragons2", tetragons2)
np.savetxt("pentagons2", pentagons2)